# 1. Bibliotecas

In [0]:
from datetime import datetime
from pyspark.sql.functions import date_format, udf, sum, col, lit, month
from pyspark.sql.types import StringType, DecimalType, DateType

---
# 2. Carregando Delta Tables


## 2.1 Products

In [0]:
df_products = spark.table('datum.silver.olist_products')

## 2.2 Orders

In [0]:
df_oders = spark.table('datum.silver.olist_orders')

## 2.3 Payments

In [0]:
df_payments = spark.table('datum.silver.olist_payments')

## 2.4 Customer

In [0]:
df_customers = spark.table('datum.silver.olist_customers')

---
# 3. Gerando agrupamento dos dados para estatísticas de vendas

## 3.1 Total (R$) de vendas por dia da semana

In [0]:
dict_week_day = {
    'Sun': 'Domingo',
    'Mon': 'Segunda',
    'Tue': 'Terça',
    'Wed': 'Quarta',
    'Thu': 'Quinta',
    'Fri': 'Sexta',
    'Sat': 'Sábado',
}

udf_day_of_week = udf(lambda x: dict_week_day.get(str(x)))

In [0]:
# Unindo os DataFrames de orders e payment para pegarmos as datas e valores nelas para agruparmos posteriormente
df_vendas_dia_semana = df_oders.select('order_id', 'order_approved_at').where(col('order_approved_at').isNotNull())\
                               .join(
                                   df_payments.select('order_id', 'payment_value').where(col('payment_value').isNotNull()), 
                                   on = ['order_id'], how='inner')

# Pegando o dia da semana referente a data                               
df_vendas_dia_semana = df_vendas_dia_semana.withColumn('dia_da_semana', date_format('order_approved_at', 'E'))\
                                           .select('dia_da_semana', 'payment_value')

# Aplicando a função para traduzir os dias da semana para português
df_vendas_dia_semana = df_vendas_dia_semana.withColumn('dia_da_semana', udf_day_of_week('dia_da_semana'))

# Agrupando o DF nos dias da semana e somando os valor e por fim ordenando de forma decrescente por valor total
df_vendas_dia_semana = df_vendas_dia_semana.groupBy('dia_da_semana').agg(sum('payment_value').alias('total')).orderBy('total', ascending=False)

# Convertendoo total para Decimal(10,2) por ser uma formato mais ideal dentro dos dados
df_vendas_dia_semana = df_vendas_dia_semana.withColumn('total', col('total').cast(DecimalType(10,2)))

# Por fim adicionando a coluna date_ref_carga para termos o controle no futuro em caso de registros para comparação com registros anteriormente processados
df_vendas_dia_semana = df_vendas_dia_semana.withColumn('date_ref_carga', lit(datetime.now()).cast(DateType()))

In [0]:
df_vendas_dia_semana.limit(10).display()

dia_da_semana,total,date_ref_carga
Terça,3019003.32,2024-04-14
Quarta,2526603.51,2024-04-14
Quinta,2494340.73,2024-04-14
Sexta,2366216.26,2024-04-14
Segunda,2138720.93,2024-04-14
Sábado,2001180.50,2024-04-14
Domingo,1425677.10,2024-04-14



### 3.1.1 Salvando dado na camada gold

In [0]:
%sql

USE CATALOG datum

In [0]:
%sql

USE DATABASE gold

In [0]:
%sql

CREATE TABLE IF NOT EXISTS olist_vendas_dias_semana
(
  dia_da_semana  STRING NOT NULL,
  total          DECIMAL(10,2),
  date_ref_carga DATE
)
USING DELTA
LOCATION 'abfss://unity-datum@datumunity.dfs.core.windows.net/gold/olist_vendas_dias_semana'
PARTITIONED BY(date_ref_carga)

In [0]:
if df_vendas_dia_semana.count() != 0 or df_vendas_dia_semana is not None:
    df_vendas_dia_semana.write.format('delta').mode('overwrite').save('abfss://unity-datum@datumunity.dfs.core.windows.net/gold/olist_vendas_dias_semana')
del df_vendas_dia_semana

## 3.2 Total (R$) de vendas por mês

In [0]:
dict_month = {
    '1': 'Janeiro',
    '2': 'Fevereiro',
    '3': 'Março',
    '4': 'Abril',
    '5': 'Maio',
    '6': 'Junho',
    '7': 'Julho',
    '8': 'Agosto',
    '9': 'Setembro',
    '10': 'Outubro',
    '11': 'Novembro',
    '12': 'Dezembro'
}

udf_dict_month = udf(lambda x: dict_month.get(str(x)))

In [0]:
# Unindo os DataFrames de orders e payment para pegarmos as datas e valores nelas para agruparmos posteriormente
df_vendas_mes = df_oders.select('order_id', 'order_approved_at').where(col('order_approved_at').isNotNull())\
                               .join(
                                   df_payments.select('order_id', 'payment_value').where(col('payment_value').isNotNull()), 
                                   on = ['order_id'], how='inner')

# Pegando o dia da semana referente a data                               
df_vendas_mes = df_vendas_mes.withColumn('mes', month('order_approved_at'))\
                                           .select('mes', 'payment_value')

# Aplicando a função para traduzir os dias da semana para português
df_vendas_mes = df_vendas_mes.withColumn('mes', udf_dict_month('mes'))

# Agrupando o DF nos dias da semana e somando os valor e por fim ordenando de forma decrescente por valor total
df_vendas_mes = df_vendas_mes.groupBy('mes').agg(sum('payment_value').alias('total')).orderBy('total', ascending=False)

# Convertendoo total para Decimal(10,2) por ser uma formato mais ideal dentro dos dados
df_vendas_mes = df_vendas_mes.withColumn('total', col('total').cast(DecimalType(10,2)))

# Por fim adicionando a coluna date_ref_carga para termos o controle no futuro em caso de registros para comparação com registros anteriormente processados
df_vendas_mes = df_vendas_mes.withColumn('date_ref_carga', lit(datetime.now()).cast(DateType()))

In [0]:
df_vendas_mes.limit(10).display()

mes,total,date_ref_carga
Maio,1773271.90,2024-04-14
Agosto,1708221.51,2024-04-14
Julho,1628398.06,2024-04-14
Março,1616403.88,2024-04-14
Abril,1551231.40,2024-04-14
Junho,1543311.41,2024-04-14
Fevereiro,1276258.23,2024-04-14
Janeiro,1237912.54,2024-04-14
Novembro,1174911.51,2024-04-14
Dezembro,902554.31,2024-04-14



### 3.2.1 Salvando dado na camada gold

In [0]:
%sql

CREATE TABLE IF NOT EXISTS olist_vendas_mes
(
  mes            STRING NOT NULL,
  total          DECIMAL(10,2),
  date_ref_carga DATE
)
USING DELTA
LOCATION 'abfss://unity-datum@datumunity.dfs.core.windows.net/gold/olist_vendas_mes'
PARTITIONED BY(date_ref_carga)

In [0]:
if df_vendas_mes.count() != 0 or df_vendas_mes is not None:
    df_vendas_mes.write.format('delta').mode('overwrite').save('abfss://unity-datum@datumunity.dfs.core.windows.net/gold/olist_vendas_mes')
del df_vendas_mes

## 3.3 Total (R$) de vendas por estado

In [0]:
df_vendas_estado = df_oders.join(df_customers, on = ['customer_id'], how='inner')\
                           .select('order_id', 'customer_state')\
                           .join(df_payments.select('order_id', 'payment_value'), 
                                 on = ['order_id'], 
                                 how='inner').\
                           select('customer_state', 'payment_value')

df_vendas_estado = df_vendas_estado.groupBy('customer_state').agg(sum('payment_value').alias('total'))
df_vendas_estado = df_vendas_estado.withColumnRenamed('customer_state', 'estado')
df_vendas_estado = df_vendas_estado.withColumn('total', col('total').cast(DecimalType(10,2)))
df_vendas_estado = df_vendas_estado.withColumn('date_ref_carga', lit(datetime.now()).cast(DateType()))

In [0]:
df_vendas_estado.orderBy('total', ascending=False).limit(10).display()

estado,total,date_ref_carga
SP,5998226.96,2024-04-14
RJ,2144379.69,2024-04-14
MG,1872257.26,2024-04-14
RS,890898.54,2024-04-14
PR,811156.38,2024-04-14
SC,623086.43,2024-04-14
BA,616645.82,2024-04-14
DF,355141.08,2024-04-14
GO,350092.31,2024-04-14
ES,325967.55,2024-04-14



### 3.3.1 Salvando dado na camada gold

In [0]:
%sql

CREATE TABLE IF NOT EXISTS olist_vendas_estado
(
  estado         STRING NOT NULL,
  total          DECIMAL(10,2),
  date_ref_carga DATE
)
USING DELTA
LOCATION 'abfss://unity-datum@datumunity.dfs.core.windows.net/gold/olist_vendas_estado'
PARTITIONED BY(date_ref_carga)

In [0]:
if df_vendas_estado.count() != 0 or df_vendas_estado is not None:
    df_vendas_estado.write.format('delta').mode('overwrite').save('abfss://unity-datum@datumunity.dfs.core.windows.net/gold/olist_vendas_estado')
del df_vendas_estado